In [10]:
#################################  SELECTIONS   ###########################################
make = "FORD"                                # "" for no make selected
###########################################################################################

In [11]:
import warnings
warnings.filterwarnings("ignore")

In [12]:
import pandas as pd
pd.set_option('display.max_colwidth', None)
import numpy as np

In [13]:
from nltk.tokenize import word_tokenize
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
from nltk.corpus import stopwords
stopword = nltk.corpus.stopwords.words('english')
from nltk.stem.wordnet import WordNetLemmatizer

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/gianluigi/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/gianluigi/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /Users/gianluigi/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [14]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [15]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

In [16]:
# import boto3
# import s3fs

In [17]:
import re
import string
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [18]:
from sentence_transformers import SentenceTransformer
from sentence_transformers import models
from sentence_transformers import SentenceTransformer, SentencesDataset, InputExample, losses
from torch.utils.data import DataLoader

In [19]:
colnames = ["BULNO", "BULREP", "ID", "BULDTE", "COMPNAME", "MAKETXT", "MODELTXT", "YEARTXT", "DATEA", "SUMMARY"]

df0 = pd.read_csv('Datasets/FLAT_TSBS.csv', sep='\t|\r|\n', encoding='iso-8859-1', header=None, names=colnames)

In [20]:
df0['KEY'] = range(0, 0+len(df0)) 

In [21]:
df1 = df0[["KEY","BULNO", "BULREP", "ID", "BULDTE", "COMPNAME", "MAKETXT", "MODELTXT", "YEARTXT", "DATEA", "SUMMARY"]]

In [22]:
df1 = df1.drop("BULREP", axis=1)            # Dropping column with many null values

In [23]:
data_types_dict = {"KEY":str,"BULNO":str, "ID":str, "COMPNAME":str,"BULDTE":str, "COMPNAME":str, 
                   "MAKETXT":str, "MODELTXT":str, "YEARTXT":str, "DATEA":str, "SUMMARY":str}
df1 = df1.astype(data_types_dict) 

In [24]:
print("Total Number of Bulletins in the Dataset:", len(df1))

Total Number of Bulletins in the Dataset: 2876141


In [25]:
df2 = df1.drop_duplicates(subset="SUMMARY")     # Dropping bulletins with identical summary

In [26]:
print("Total Number of Bulletins in the Dataset (no duplicates):", len(df2))

Total Number of Bulletins in the Dataset (no duplicates): 130627


In [27]:
if len(make) > 0:
    df2 =df2[df2["MAKETXT"] == make]
else:
    print("No Make Selected")

In [28]:
base_document_key = '1697356'           # this is just an example

print("Make Selected:", make)

print("Total Number of Bulletins in Evaluation:", len(df2))


Make Selected: FORD
Total Number of Bulletins in Evaluation: 9101


In [29]:
df3 = df2.copy()          # keeping a copy of the dataset before preprocessing data

In [30]:
# Pre-process data

In [31]:
Column_Name = 'SUMMARY'
Clean_Column_Name = 'SUMMARY_CLEAN'
df3[Clean_Column_Name] =df3[Column_Name]

In [32]:
def remove_punctuation(text):
    string.punctuation = string.punctuation.replace('-', '')
    no_punct=[words for words in text if words not in string.punctuation]
    words_wo_punct=''.join(no_punct)
    return words_wo_punct

In [33]:
df3[Clean_Column_Name] = df3[Clean_Column_Name].apply(lambda x: remove_punctuation(x))

In [34]:
def remove_numbers(text):
    no_num = [word for word in text if not word.isdigit()]
    words_no_num = ''.join(no_num)
    return words_no_num

In [35]:
df3[Clean_Column_Name] = df3[Clean_Column_Name].apply(lambda x: remove_numbers(x))

In [36]:
def tokenize(text):
    split=re.split("\W+",text) 
    return split

In [37]:
df3[Clean_Column_Name]=df3[Clean_Column_Name].apply(lambda x: tokenize(x.lower()))

In [38]:
def remove_stopwords(text):
    text=[word for word in text if word not in stopword]
    return text

In [39]:
# df3[Clean_Column_Name] = df3[Clean_Column_Name].apply(lambda x: remove_stopwords(x))

In [40]:
def lemmatize(text):
    lem = WordNetLemmatizer()
    lemma = [lem.lemmatize(word) for word in text] 
    lemmatized_text = " ".join(lemma)
    return lemmatized_text

In [41]:
df3[Clean_Column_Name] = df3[Clean_Column_Name].apply(lambda x: lemmatize(x))

In [42]:
df = df3.copy()

In [43]:
df3.to_csv('ServiceFiles/data_processed.csv', index=False)

In [44]:
# Setup the environment and import processed data

In [45]:
df = pd.read_csv('ServiceFiles/data_processed.csv')
df['SUMMARY_CLEAN'] = df['SUMMARY_CLEAN'].astype(str)

In [46]:
# Select make (Opt.), base document, and number of documents to analyze

In [47]:
if len(make) > 0:
    df =df[df["MAKETXT"] == make]
else:
    print("No Make Selected")

In [48]:
documents_df = df["SUMMARY_CLEAN"].values

In [49]:
# Train model

In [50]:
print('Train doc vectors')

Train doc vectors


In [51]:
tagged_data = [TaggedDocument(words=word_tokenize(doc), tags=[i]) for i, doc in enumerate(documents_df)]

model = Doc2Vec(vector_size=100,epochs=10, alpha=0.025, min_count=1)
  
model.build_vocab(tagged_data)

model.train(tagged_data,
                total_examples=model.corpus_count,
                epochs=model.epochs)

In [52]:
document_embeddings=np.zeros((documents_df.shape[0],100))

for i in range(len(document_embeddings)):
    document_embeddings[i]=model.docvecs[i]
    
    
pairwise_similarities = cosine_similarity(document_embeddings)


In [53]:
doc2vec = pd.DataFrame(document_embeddings)

In [54]:
doc2vec.to_csv('ServiceFiles/doc2vec.csv', index=False)

In [55]:
print('Done with doc vectors')

Done with doc vectors


In [56]:
# %run bert_embedding.ipynb

In [57]:
word_embedding_model = models.Transformer('bert-base-uncased', max_seq_length=256)
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension())

model = SentenceTransformer(modules=[word_embedding_model, pooling_model])

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [58]:
df = pd.read_csv('ServiceFiles/data_processed.csv')
doc2vec = pd.read_csv('ServiceFiles/doc2vec.csv')

In [59]:
csm = cosine_similarity(doc2vec)

my_dict = {}
for i in range(len(csm)):
    for j, elem in enumerate(csm[i]):
        if(elem) and (j, i) not in my_dict:
            my_dict[(i,j)] = elem
            

In [60]:
len(my_dict)

41418651

In [61]:
dd = pd.DataFrame.from_dict(my_dict,orient='index')
dd.index = pd.MultiIndex.from_tuples(dd.index)

In [62]:
dd.to_csv('ServiceFiles/doc2vec_csm_values.csv', index=False)

In [63]:
csm_sample = dd   # employ to use less vectors

In [64]:
csm_sample.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 41418651 entries, (0, 0) to (9100, 9100)
Data columns (total 1 columns):
 #   Column  Dtype  
---  ------  -----  
 0   0       float64
dtypes: float64(1)
memory usage: 474.6 MB


In [65]:

train_examples = []
for idx, summary in zip(csm_sample, df): 
    
    train_examples.append(InputExample(texts=[str(df['SUMMARY_CLEAN'].to_frame().loc[csm_sample.index.get_level_values(0).values]), str(df['SUMMARY_CLEAN'].to_frame().loc[csm_sample.index.get_level_values(1).values])], label = csm_sample.values))

In [66]:
print('Begin training BERT')


#Define the model. Either from scratch of by loading a pre-trained model
model = SentenceTransformer('nli-bert-large')

#Define your train dataset, the dataloader and the train loss
train_dataset = SentencesDataset(train_examples, model)
train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=16)
train_loss = losses.BatchAllTripletLoss(model)

#Tune the model
model.fit(train_objectives=[(train_dataloader, train_loss)], epochs=10, warmup_steps=100)

Begin training BERT


Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1 [00:00<?, ?it/s]

...

Iteration:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1 [00:00<?, ?it/s]

In [67]:
augmented_embeddings = pd.DataFrame(model.encode(df['SUMMARY_CLEAN']))

In [68]:
augmented_embeddings.to_csv('ServiceFiles/bert_tuned_embedding.csv',index=False)

In [69]:
print('End training BERT')

End training BERT


In [70]:
documents_df = df["SUMMARY_CLEAN"].values

In [71]:
# Train model

In [72]:
embeddings = pd.read_csv('ServiceFiles/bert_tuned_embedding.csv')

In [73]:
# Generate similarity matrix

In [74]:
pairwise_similarities=cosine_similarity(embeddings)

In [75]:
pairwise_similarity_matrix = pd.DataFrame(pairwise_similarities, columns=df.iloc[:,0], index=df.iloc[:,0] )

In [76]:
# Show results

In [77]:
dd1 = df.sample(3,random_state=0)
sample_docs = dd1['KEY'].to_numpy()
print(sample_docs)

[1909899  338766  386091]


In [78]:
report_all =  pd.DataFrame(columns=["Key","Bulletin Number", "Make" , "Model", "Year", "Summary"])

for i in sample_docs:

    base_document_key = i

    results = pd.DataFrame(pairwise_similarity_matrix.loc[:,base_document_key]
                       .sort_values(axis=0, ascending=False))

    results10 = results.iloc[0:11,:]

    results10 = results10.reset_index()

    bul_list = results10['KEY'].to_numpy()

    myList = []
    

    for comparison_doc in bul_list:
        Key = df.loc[df["KEY"] == comparison_doc].KEY.to_string(index=False)
        Bulletin_Number = df.loc[df["KEY"] == comparison_doc].BULNO.to_string(index=False)
        Make = df.loc[df["KEY"] == comparison_doc].MAKETXT.to_string(index=False)
        Model = df.loc[df["KEY"] == comparison_doc].MODELTXT.to_string(index=False)
        Year = df.loc[df["KEY"] == comparison_doc].YEARTXT.to_string(index=False)
        Summary = df.loc[df["KEY"] == comparison_doc].SUMMARY.to_string(index=False)
        myList.append([Key, Bulletin_Number,Make,Model, Year, Summary])

    report = pd.DataFrame(myList, columns=["Key","Bulletin Number", "Make" , "Model", "Year", "Summary"])
    
    report_all = report_all.append(report)

In [79]:
report_all.to_html('report0.html')

In [80]:
# import os
# try:
#    os.remove("ServiceFiles/data_processed.csv")
#    os.remove("ServiceFiles/doc2vec.csv")
#    os.remove("ServiceFiles/doc2vec_csm_values.csv")
#    os.remove("ServiceFiles/bert_tuned_embedding.csv")
# except OSError as error: 
#    print(error) 

In [81]:
model

SentenceTransformer(
  (0): Transformer(
    (auto_model): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(30522, 1024, padding_idx=0)
        (position_embeddings): Embedding(512, 1024)
        (token_type_embeddings): Embedding(2, 1024)
        (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): BertEncoder(
        (layer): ModuleList(
          (0): BertLayer(
            (attention): BertAttention(
              (self): BertSelfAttention(
                (query): Linear(in_features=1024, out_features=1024, bias=True)
                (key): Linear(in_features=1024, out_features=1024, bias=True)
                (value): Linear(in_features=1024, out_features=1024, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): BertSelfOutput(
                (dense): Linear(in_features=1024, out_features=1024, bias=T